In [1]:
import numpy as np
import json
from pymongo import MongoClient
import torch
import pandas as pd

client = MongoClient()
db = client['gtr']

In [2]:
def sort_to_k(ary,k,key=lambda x:x,reversed=False):
    k = min(k,len(ary))
    for i in range(k):
        for j in range(len(ary)-1-i):
            if not reversed:
                if key(ary[len(ary)-1-j]) < key(ary[len(ary)-2-j]):
                    ary[len(ary)-1-j],ary[len(ary)-2-j] = ary[len(ary)-2-j],ary[len(ary)-1-j]
            else:
                if key(ary[len(ary)-1-j]) > key(ary[len(ary)-2-j]):
                    ary[len(ary)-1-j],ary[len(ary)-2-j] = ary[len(ary)-2-j],ary[len(ary)-1-j]
    return ary

In [3]:
repo_core_teams = {}
for doc in db['repo_core_targets'].find():
    repo_core_teams[doc['repo']] = doc['core_team']

In [4]:
repo_profiles = {}
for doc in db['repo_profiles'].find()[:10]:
    if not doc['repo'] in repo_core_teams:
        continue
    repo_profiles[doc['repo']] = doc
repo_profiles_df = pd.DataFrame(repo_profiles,index={'size','forks','subscribers','watchers','languages','topics'}).transpose()
repo_profiles_df.fillna('',inplace=True)
print(repo_profiles_df.iloc[0])

languages      [FreeMarker, Gherkin, Scilab, Java]
topics                [aws-sdk, java, aws, amazon]
watchers                                      2781
forks                                         2071
size                                       1082924
subscribers                                    331
Name: aws/aws-sdk-java, dtype: object


In [5]:
user_profiles = {}
for doc in db['user_profiles'].find()[:]:
    user_profiles[doc['user']] = doc
user_profiles_df = pd.DataFrame(user_profiles,index={'repo_size','repo_forks','repo_subscribers','repo_watchers','languages','topics'}).transpose()
user_profiles_df.rename(columns={
                                   'repo_size':'size',
                                   'repo_forks':'forks',
                                   'repo_subscribers':'subscribers',
                                   'repo_watchers':'watchers',
                                   'languages':'languages',
                                   'topics':'topics'
                               },inplace=True)
user_profiles_df.fillna('',inplace=True)
print(user_profiles_df.iloc[0])

languages      [QMake, PHP, Java, Python, PLpgSQL, CSS, Go, R...
topics         [python27, gulp, python35, wordpress, arabic, ...
subscribers                                               32.875
watchers                                                 510.833
forks                                                    99.0417
size                                                     16778.4
Name: 01walid, dtype: object


In [6]:
numerics = ['size','forks','subscribers','watchers']
non_numerics = ['languages','topics']

minn = repo_profiles_df[numerics].min()
maxx = repo_profiles_df[numerics].max()
repo_profiles_df[numerics] = (repo_profiles_df[numerics]-minn)/(maxx-minn)
user_profiles_df[numerics] = (user_profiles_df[numerics]-minn)/(maxx-minn)

In [7]:
user_teams = {}
for doc in db['team_profiles'].find():
    tm = doc['team']
    contris = sum(doc['member_contributions'].values())
    degrees = sum(doc['member_degrees'].values())
    for user in json.loads(tm):
        if not user in user_teams:
            user_teams[user] = {}
        user_teams[user][tm] = {
            'contri':doc['member_contributions'][user]/contris,
            'degree':doc['member_degrees'][user]/degrees
        }

In [8]:
def euclidean_non_numerics(p1,p2):
    if p1[0] and p2[0]:
        c = len(set(p1[0]).intersection(set(p2[0])))
        dis_langs = 1-c/(len(p1[0])+len(p2[0])-c)
    else:
        dis_langs = 0
    if p1[1] and p2[1]:
        c = len(set(p1[1]).intersection(set(p2[1])))
        dis_topics = 1-c/(len(p1[1])+len(p2[1])-c)
    else:
        dis_topics = 1
    return dis_langs**2 + dis_topics**2
euclidean_non_numerics_v =  np.vectorize(euclidean_non_numerics,signature="(n),(n)->()")
print(euclidean_non_numerics_v(repo_profiles_df.iloc[0][non_numerics],user_profiles_df[non_numerics]))

[1.88581315 2.         1.90702948 ... 2.         1.87513007 2.        ]


In [9]:
cuda0 = torch.device('cuda:0')
user_num = torch.tensor(user_profiles_df[numerics].values,device=cuda0,dtype=torch.float16,requires_grad=False)
batch_size = 10000
dis_num = []
for i in range(repo_profiles_df.shape[0]//batch_size+1):
    print(i)
    r_n = torch.tensor(repo_profiles_df.iloc[i*batch_size:(i+1)*batch_size][numerics].values,device=cuda0,dtype=torch.float16,requires_grad=False)
    d_n = (r_n**2).sum(1,keepdim=True)+(user_num**2).sum(1,keepdim=True).transpose(0,1)-2*r_n.matmul(user_num.transpose(0,1))
    del r_n
    dis_num.extend(d_n.cpu())
    del d_n
print(len(dis_num))

0
10


In [18]:
import time

start = time.time()
cnt = 0
k = 50
for repo,repo_profile in repo_profiles_df.iterrows():
    print(cnt)
    dis_non_num = torch.tensor(euclidean_non_numerics_v(repo_profile[non_numerics],user_profiles_df[non_numerics]),device=cuda0,dtype=torch.float16,requires_grad=False)
    dis = dis_num[cnt-1].cuda(device=cuda0)+dis_non_num
    dis = dis.cpu().numpy()
    team_scores = {}
    team_score_contri = {}
    team_score_degree = {}
    for i,user in enumerate(user_profiles_df.index):
        for tm in user_teams[user]:
            if not tm in team_scores:
                team_scores[tm] = []
                team_score_contri[tm] = 0
                team_score_degree[tm] = 0
            team_scores[tm].append(dis[i])
            team_score_contri[tm] += user_teams[user][tm]['contri']
            team_score_degree[tm] += user_teams[user][tm]['degree']
    team_score_min = {}
    team_score_mean = {}
    team_score_max = {}
    for tm in team_scores:
        team_score_min[tm] = min(team_scores[tm])
        team_score_max[tm] = max(team_scores[tm])
        team_score_mean[tm] = sum(team_scores[tm])/len(team_scores[tm])
    tms_min = sort_to_k(list(team_score_min),k,key=lambda i:team_score_min[i])
    tms_mean = sort_to_k(list(team_score_mean),k,key=lambda i:team_score_mean[i])
    tms_max = sort_to_k(list(team_score_min),k,key=lambda i:team_score_max[i])
    tms_contri = sort_to_k(list(team_score_min),k,key=lambda i:team_score_contri[i])
    tms_degree = sort_to_k(list(team_score_min),k,key=lambda i:team_score_degree[i])
    print(repo)
    for tm in tms_min[:k]:
        print(tm,float(team_score_min[tm]))
    for tm in tms_mean[:k]:
        print(tm,team_score_mean[tm])
    for tm in tms_max[:k]:
        print(tm,team_score_max[tm])
    for tm in tms_contri[:k]:
        print(tm,team_score_contri[tm])
    for tm in tms_degree[:k]:
        print(tm,team_score_degree[tm])
        
    cnt += 1
print(time.time()-start)

0
aws/aws-sdk-java
["shorea", "dagnir", "fernomac", "varunnvs92", "kiiadi", "spfink", "millems", "zoewangg"] 0.90234375
["rhyzx", "xpepermint", "yyx990803", "fundon", "rayfranco", "simplesmiler", "brillout", "dwightjack", "OEvgeny", "syropian", "netpi", "finico", "ludo237", "kikyous", "fergaldoyle", "iJackUA", "RashadSaleh", "eljefedelrodeodeljefe", "SmirnovW", "SimulatedGREG", "Twiknight", "Saymon-biz", "surmon-china", "kevinongko", "iatsiuk"] 0.94970703125
["afollestad", "david-yz-liu", "Ian-Stewart-Binks", "christinem", "SoftwareDev", "munrocape", "arkon", "reidka", "jwebbed", "GitGeminiano", "qasim", "akbiggs", "alexvail", "lvwrence", "kabouzeid", "adriancampos", "DVDAndroid", "WChrisK", "TheDorkKnightRises", "PkHutch", "RyanDJLee", "b2simms", "MBAustin", "VictorBG", "theduffmaster", "tkashkin", "doompadee", "amrrsharaff"] 0.9873046875
["n9ti", "TonPC64", "pangaunn", "yutita", "miw1513", "biigpongsatorn", "nuttakit2405", "Paretkf"] 1.0
["eyalzek", "nitmir", "zed", "satwikkansal", "

["willianjusten", "davidsonfellipe", "kogarashisan", "felipemrodrigues", "guilouro", "michelamaral", "mvfsillva", "cjpatoilo"] 1.0234375
["spesnova", "awakia", "RoxasShadow", "inakiabt", "starandtina", "larrylv", "kevingo", "donnemartin", "OlafvdSpek", "bengadbois", "MaximAbramchuck", "bryanjenningz", "koudaiii", "zanemayo", "ralphite", "SvistunovYuriy", "csiki", "luisbg", "fluency03", "cciollaro", "yaoxiaotong", "ianpark", "iamsidshetty", "jpcaparas", "balki-server", "mad01", "Gaafar", "baldassarreFe", "jxub", "tangkangkai", "CharlesSantiago", "tsukukobaan", "rishabh115", "hakaari"] 1.0234375
["leviding", "EmilyQiRabbit", "sunhaokk", "7Ethan", "Starriers", "sisibeloved"] 1.02734375
["markdalgleish", "nicolassenechal", "bbenezech", "richardPFisk", "ryardley", "jahredhope", "mengtzu", "LeviSeek", "mattsjones", "seangrant", "hscgavin", "KimDal-hyeong", "lgshaw"] 1.0283203125
["mitogh", "talyssonoc", "davidfurlong", "pbassut", "juanfra", "vasanthk", "ryanmcdermott", "jordalgo", "DavidVuji

["dossorio", "kindraywind", "thovo", "hienvd", "marceloogeda", "letientai299", "Kiricon", "jwasham", "SaurabhMhaisekar", "dangkhoasdc", "AnimeshShaw", "sjaramillo10", "amoulkaf", "dapp1990", "YanaBe", "alangarciar", "wilkoklak", "andersontr15", "avizmarlon", "AmrReda", "duytruong", "iphayao", "jarvisluong", "YoSaucedo", "meshuamam"] 1.6530078125
["patxu", "annieke"] 1.6767578125
["piemonte", "olucurious", "tesths", "othreecodes", "alexiscn", "TBXark", "xavipedrals", "mustafaatalar"] 1.6910400390625
["ashishfarmer", "petrex", "rohithkrn"] 1.7190755208333333
["petejohanson", "griest024", "RobertSmits", "pgbross", "stasson"] 1.721875
["chickenlj", "manzhizhen", "tuser4198", "diecui1202", "vangoleo", "cvictory", "ningyu1", "ZhichX", "xgqiuq"] 1.7273220486111112
["thaidn", "bleichen"] 1.7275390625
["elisherer", "anescobar1991", "thomasbertet"] 1.7333984375
["leviding", "EmilyQiRabbit", "sunhaokk", "7Ethan", "Starriers", "sisibeloved"] 1.7340494791666667
["vigzmv", "TheoVitkovskiy"] 1.734863

["aledbf", "jstoja", "dwelch2344", "mustafaakin", "dpratt", "cgurnik", "bfin", "Gregy", "vood", "arjanschaaf", "glennslaven", "weitzj", "craigmonson", "coryfklein", "amincheloh", "lcharkiewicz", "morganwu277", "acoshift", "ZhuPeng", "yowenter", "victortrac", "distol", "RemingtonReackhof", "rafik8", "jrynyt", "lichen2013", "lkirill", "aviadatsnyk", "qiujian16", "pwillie", "canhnt", "llb4ll", "jonaskint", "nevetS", "chenquanzhao", "Rajadeepan", "thedch", "unthought", "ctriv", "nusx", "iamgd67", "chrisblu", "f84anton", "ryanmckaytx", "jrthrawny", "hxquangnhat", "qiupeng-huacloud", "turettn", "yulng", "liticer", "frandbf", "dsalamancaMS", "liyongxin"] 0.9999999999999991
["pivanov", "phil-brown", "rogerhu", "neverov", "adepue", "flovilmart", "marvelm", "KoCMoHaBTa", "Jawnnypoo", "emadehsan", "dtsolis", "theill", "montymxb", "vferdiansyah", "charliewilliams", "Bragegs", "deashay", "Rexee", "maninga", "eduardbosch", "6thfdwp", "cipiripper", "valeryvaskabovich", "nbolatov", "conghoang", "bamwa

["cytec", "fernandog", "abeloin", "markheloking", "lucianot54", "TagForce", "gborri", "josh4trunks", "echel0n", "Tawmmm", "sn3akz", "nervling", "Captsha", "coolius", "vzoltan", "thermodyn", "ThePieMan", "vandalon", "swamp12", "Deddiekoel", "Benouch", "dsoms", "alaorath", "iguyking", "andofrjando", "d1b", "joshjowen", "mwarnaar", "lsborg", "BillNasty", "Arcanemagus", "SickrageDK", "luxmoggy", "Wally73", "aidik", "joshuahiggins", "cptjhmiller", "EchelonFour", "Sparhawk76", "InfernoZeus", "vanackej", "Eiber", "daarcm", "dgholz", "DocMarty84", "bjornhamra", "MaximusCub", "ElmerLastdrager", "sy6502a", "HenrikPoulsen", "HakShak", "normtown", "Sylrob434", "stefaang", "bdoner", "cswelin", "p00chie"] 0.9999999999999993
["alphaleonis", "iislucas", "vvakame", "nestalk", "jtmueller", "mickdelaney", "staxmanade", "bluong", "martinmcwhorter", "pushplay", "smithkl42", "flcdrg", "nycdotnet", "misak113", "craigktreasure", "jpevarnek", "lokeshpeta", "xperiments", "ryiwamoto", "SergioMorchon", "kanreisa"

...

["juliankrispel", "tagliala", "gurisko", "sugarshin", "libc", "supercodepoet", "woahdae", "theonegri", "Rosey", "dlmiles", "robmadole", "tagCincy", "Vusys", "eelkeblok", "bsara", "theoreticaLee", "prayogoa", "MacroChip", "Kornil"] 0.9999999999999994
["gaearon", "skidding", "NiGhTTraX", "mderazon", "smmoosavi", "arjunblj", "YoruNoHikage", "saimonmoore", "davidagraf", "adam-beck", "smashercosmo", "hkal", "tonyspiro", "Reanmachine", "danmartinez101", "n1ghtmare", "zebapy", "pekala", "rwillrich", "MrHus", "catalinmiron", "ConAntonakos", "vitalybe", "chrislaughlin", "romseguy", "ms88privat", "petehouston", "micha149", "rchanou", "ghondar", "monochrome-yeh", "sverrejoh", "AndrewHenderson", "sashashakun", "Alex-ray", "rgrochowicz", "dannycalleri", "aleburato", "Urthen", "haohcraft", "diligiant", "zeel", "breaddevil", "fezhengjin", "dceddia", "GautierT", "borela", "esturcke", "lopezator", "BDav24", "yenshih", "ogawa0071", "krinoid", "kafein", "filiphosko", "ApacheEx", "jayair", "exiadbq", "Hot

["afc163", "yesmeck", "TossShinHwa", "viztor", "shenqihui", "int64ago", "taviroquai", "williamkhshea", "abenhamdine", "RaoHai", "kirbo", "kagd", "djyde", "Alex1990", "miaojiuchen", "picodoth", "tvasenin", "benjycui", "plandem", "snadn", "ystarlongzi", "matthewlein", "timarney", "manjitkumar", "cdeutsch", "zheeeng", "queimadus", "madisvain", "JesperWe", "wuzhao", "chentony", "yutingzhao1991", "jaredleechn", "muzea", "Kamahl19", "pnghai", "jackple", "LeuisKen", "ChrisFan", "MrPeak", "edgji", "sosz", "ddcat1115", "mzohaibqc", "novi", "koobitor", "lixiaoyang1992", "kossel", "Attrash-Islam", "wSLecHayfIeNdock", "aaronplanell", "Nokecy", "sjy", "khalibloo", "dsgkirkby", "ziaochina", "farskid", "Frezc", "diegourban", "menberg", "KgTong", "valleykid", "acamposruiz", "Rohanhacker", "mgrdevport", "twisger", "blkdr", "chj-damon", "hengkx", "LeeHarlan", "kocielnik", "weihong1028", "delesseps", "danedavid", "yociduo", "ahalimkara", "dilidili", "costagolub", "kanweiwei", "EscoCreeks", "happier2", "j

["ataraxus", "sankargorthi", "log0ymxm", "chubinou", "JackWink", "wonderslug", "soulfly", "ferristseng", "philicious", "bluengreen", "Glagnar", "wing328", "ianjoneill", "RobertBiehl", "hideya", "toomuchpete", "beatcracker", "dpeschman", "aersamkull", "kartenkarsten", "y-higuchi", "brianvoss", "frinux", "keysolutions", "nazieb", "Articus", "Edubits", "SparrowJang", "pamuche", "tomekc", "F481", "julienfouilhe", "BastiOfBerlin", "wwwjfy", "laurentvdl", "mtngld", "kolyjjj", "gwen15", "zxy198717", "shawkinsl", "ampem", "apolci", "kkrauth", "rostskadat", "ajainy", "akajla09", "jgavris", "justusth", "thibaultclem", "pculligan", "rbpasker", "jaz-ah", "davebaird", "woz5999", "mtakaki", "ww-mgr", "sugi", "GriffinSchneider", "jerry1621", "dhontecillas", "bmsantos", "ahmettahasakar", "wolfAle", "giovannibotta", "joeboyscout04", "neoasket", "vzeman", "nrekretep", "abcsun", "mmschettler", "JMDHo", "hcwilhelm", "kesh92", "andriydruk", "dmytrobr", "mbarclay", "zhenjun115", "kim-sondrup", "qtstc", "sal

["wobh", "kytrinyx", "tstirrat15", "masters3d", "hankturowski", "allcentury", "LegalizeAdulthood", "drueck", "simonjefford", "jendiamond", "mdesantis", "brettchalupa", "RickWieman", "seejohnrun", "ohookins", "ZacharyRSmith", "lhoBas", "snahor", "spazm", "scottwhudson", "groyoh", "sit", "nickkjordan", "vguerra", "davearonson", "aarti", "connorlay", "bcc32", "ebengt", "amar47shah", "kickinbahk", "DuBistKomisch", "sarahjschultz", "mike-hewitson", "claytonflesher", "junedev", "AJEzk", "laurafeier", "ecrmnn", "oalbe", "AlexWheeler", "amscotti", "ramyaravindranath", "sdavids13", "betegelse", "sandhub", "drewbs", "JPBetley", "aranair", "CRivasGomez", "celiagomezdev", "jdsutherland", "thedevelopnik", "Rahmeen14", "m-sameer", "kchenery", "yilkalargaw"] 0.9999999999999987
["orta", "iosdev-republicofapps", "soffes", "tamastimar", "fabiomassimo", "caffeineflo", "natan", "sgtsquiggs", "jacobwgillespie", "pbendersky", "aksswami", "michalkonturek", "AndyChenW", "rbsgn", "rhodgkins", "tabuchid", "imak

["jaroslawr", "tarsius", "proofit404", "ahungry", "shackra", "katspaugh", "kiwanami", "eschulte", "holocronweaver", "myuhe", "bsuh", "takaxp", "gabesoft", "johanclaesson", "osener", "uu1101", "garyo", "dangirsh", "bastianbeischer", "baohaojun", "ytaka", "iRi-E", "khiker", "xwl", "rubikitch", "decoursin", "loveshack", "BinaryKhaos", "larsmagne", "naclander", "swflint", "jtatarik", "dellison", "dajva"] 0.9999999999999988
["x64x6a", "abpolym", "vvvverre", "pawlos", "boogy", "niklasb", "dhanvi", "wapiflapi", "bcyrill", "peter50216", "evandrix", "denwilly", "redknight99", "cryptax", "tyage", "superkojiman", "RyanAiden", "derlaft", "mrexcessive", "mrt-prodz", "gaffe23", "stefan2904", "ZetaTwo", "boaster", "mheistermann", "tomvangoethem", "fbonzon", "elliptic-shiho", "pe3zx", "JonathanSinger", "f0rki", "unicornsasfuel", "LiveOverflow", "chadillac", "jagotu", "L4ys", "ctfhacker", "vito-lbs", "x0w1", "tothi", "Matir"] 0.9999999999999988
["wm4", "bucaneer", "Nikoli", "quilloss", "teohhanhui", "B

In [20]:
import time

start = time.time()
cnt = 0
k = 50
for repo,repo_profile in repo_profiles_df.iterrows():
    print(cnt)
    dis_non_num = np.array(euclidean_non_numerics_v(repo_profile[non_numerics],user_profiles_df[non_numerics]),dtype=np.float16)
    dis = dis_num[cnt-1].numpy()+dis_non_num
#     dis = dis.numpy()
    team_scores = {}
    team_score_contri = {}
    team_score_degree = {}
    for i,user in enumerate(user_profiles_df.index):
        for tm in user_teams[user]:
            if not tm in team_scores:
                team_scores[tm] = []
                team_score_contri[tm] = 0
                team_score_degree[tm] = 0
            team_scores[tm].append(dis[i])
            team_score_contri[tm] += user_teams[user][tm]['contri']
            team_score_degree[tm] += user_teams[user][tm]['degree']
    team_score_min = {}
    team_score_mean = {}
    team_score_max = {}
    for tm in team_scores:
        team_score_min[tm] = min(team_scores[tm])
        team_score_max[tm] = max(team_scores[tm])
        team_score_mean[tm] = sum(team_scores[tm])/len(team_scores[tm])
    tms_min = sort_to_k(list(team_score_min),k,key=lambda i:team_score_min[i])
    tms_mean = sort_to_k(list(team_score_mean),k,key=lambda i:team_score_mean[i])
    tms_max = sort_to_k(list(team_score_min),k,key=lambda i:team_score_max[i])
    tms_contri = sort_to_k(list(team_score_min),k,key=lambda i:team_score_contri[i])
    tms_degree = sort_to_k(list(team_score_min),k,key=lambda i:team_score_degree[i])
    print(repo)
    for tm in tms_min[:k]:
        print(tm,float(team_score_min[tm]))
    for tm in tms_mean[:k]:
        print(tm,team_score_mean[tm])
    for tm in tms_max[:k]:
        print(tm,team_score_max[tm])
    for tm in tms_contri[:k]:
        print(tm,team_score_contri[tm])
    for tm in tms_degree[:k]:
        print(tm,team_score_degree[tm])
        
    cnt += 1
print(time.time()-start)

0
aws/aws-sdk-java
["shorea", "dagnir", "fernomac", "varunnvs92", "kiiadi", "spfink", "millems", "zoewangg"] 0.90234375
["rhyzx", "xpepermint", "yyx990803", "fundon", "rayfranco", "simplesmiler", "brillout", "dwightjack", "OEvgeny", "syropian", "netpi", "finico", "ludo237", "kikyous", "fergaldoyle", "iJackUA", "RashadSaleh", "eljefedelrodeodeljefe", "SmirnovW", "SimulatedGREG", "Twiknight", "Saymon-biz", "surmon-china", "kevinongko", "iatsiuk"] 0.94970703125
["afollestad", "david-yz-liu", "Ian-Stewart-Binks", "christinem", "SoftwareDev", "munrocape", "arkon", "reidka", "jwebbed", "GitGeminiano", "qasim", "akbiggs", "alexvail", "lvwrence", "kabouzeid", "adriancampos", "DVDAndroid", "WChrisK", "TheDorkKnightRises", "PkHutch", "RyanDJLee", "b2simms", "MBAustin", "VictorBG", "theduffmaster", "tkashkin", "doompadee", "amrrsharaff"] 0.9873046875
["n9ti", "TonPC64", "pangaunn", "yutita", "miw1513", "biigpongsatorn", "nuttakit2405", "Paretkf"] 1.0
["eyalzek", "nitmir", "zed", "satwikkansal", "

["willianjusten", "davidsonfellipe", "kogarashisan", "felipemrodrigues", "guilouro", "michelamaral", "mvfsillva", "cjpatoilo"] 1.0234375
["spesnova", "awakia", "RoxasShadow", "inakiabt", "starandtina", "larrylv", "kevingo", "donnemartin", "OlafvdSpek", "bengadbois", "MaximAbramchuck", "bryanjenningz", "koudaiii", "zanemayo", "ralphite", "SvistunovYuriy", "csiki", "luisbg", "fluency03", "cciollaro", "yaoxiaotong", "ianpark", "iamsidshetty", "jpcaparas", "balki-server", "mad01", "Gaafar", "baldassarreFe", "jxub", "tangkangkai", "CharlesSantiago", "tsukukobaan", "rishabh115", "hakaari"] 1.0234375
["leviding", "EmilyQiRabbit", "sunhaokk", "7Ethan", "Starriers", "sisibeloved"] 1.02734375
["markdalgleish", "nicolassenechal", "bbenezech", "richardPFisk", "ryardley", "jahredhope", "mengtzu", "LeviSeek", "mattsjones", "seangrant", "hscgavin", "KimDal-hyeong", "lgshaw"] 1.0283203125
["mitogh", "talyssonoc", "davidfurlong", "pbassut", "juanfra", "vasanthk", "ryanmcdermott", "jordalgo", "DavidVuji

["dossorio", "kindraywind", "thovo", "hienvd", "marceloogeda", "letientai299", "Kiricon", "jwasham", "SaurabhMhaisekar", "dangkhoasdc", "AnimeshShaw", "sjaramillo10", "amoulkaf", "dapp1990", "YanaBe", "alangarciar", "wilkoklak", "andersontr15", "avizmarlon", "AmrReda", "duytruong", "iphayao", "jarvisluong", "YoSaucedo", "meshuamam"] 1.6530078125
["patxu", "annieke"] 1.6767578125
["piemonte", "olucurious", "tesths", "othreecodes", "alexiscn", "TBXark", "xavipedrals", "mustafaatalar"] 1.6910400390625
["ashishfarmer", "petrex", "rohithkrn"] 1.7190755208333333
["petejohanson", "griest024", "RobertSmits", "pgbross", "stasson"] 1.721875
["chickenlj", "manzhizhen", "tuser4198", "diecui1202", "vangoleo", "cvictory", "ningyu1", "ZhichX", "xgqiuq"] 1.7273220486111112
["thaidn", "bleichen"] 1.7275390625
["elisherer", "anescobar1991", "thomasbertet"] 1.7333984375
["leviding", "EmilyQiRabbit", "sunhaokk", "7Ethan", "Starriers", "sisibeloved"] 1.7340494791666667
["vigzmv", "TheoVitkovskiy"] 1.734863

["aledbf", "jstoja", "dwelch2344", "mustafaakin", "dpratt", "cgurnik", "bfin", "Gregy", "vood", "arjanschaaf", "glennslaven", "weitzj", "craigmonson", "coryfklein", "amincheloh", "lcharkiewicz", "morganwu277", "acoshift", "ZhuPeng", "yowenter", "victortrac", "distol", "RemingtonReackhof", "rafik8", "jrynyt", "lichen2013", "lkirill", "aviadatsnyk", "qiujian16", "pwillie", "canhnt", "llb4ll", "jonaskint", "nevetS", "chenquanzhao", "Rajadeepan", "thedch", "unthought", "ctriv", "nusx", "iamgd67", "chrisblu", "f84anton", "ryanmckaytx", "jrthrawny", "hxquangnhat", "qiupeng-huacloud", "turettn", "yulng", "liticer", "frandbf", "dsalamancaMS", "liyongxin"] 0.9999999999999991
["pivanov", "phil-brown", "rogerhu", "neverov", "adepue", "flovilmart", "marvelm", "KoCMoHaBTa", "Jawnnypoo", "emadehsan", "dtsolis", "theill", "montymxb", "vferdiansyah", "charliewilliams", "Bragegs", "deashay", "Rexee", "maninga", "eduardbosch", "6thfdwp", "cipiripper", "valeryvaskabovich", "nbolatov", "conghoang", "bamwa

["cytec", "fernandog", "abeloin", "markheloking", "lucianot54", "TagForce", "gborri", "josh4trunks", "echel0n", "Tawmmm", "sn3akz", "nervling", "Captsha", "coolius", "vzoltan", "thermodyn", "ThePieMan", "vandalon", "swamp12", "Deddiekoel", "Benouch", "dsoms", "alaorath", "iguyking", "andofrjando", "d1b", "joshjowen", "mwarnaar", "lsborg", "BillNasty", "Arcanemagus", "SickrageDK", "luxmoggy", "Wally73", "aidik", "joshuahiggins", "cptjhmiller", "EchelonFour", "Sparhawk76", "InfernoZeus", "vanackej", "Eiber", "daarcm", "dgholz", "DocMarty84", "bjornhamra", "MaximusCub", "ElmerLastdrager", "sy6502a", "HenrikPoulsen", "HakShak", "normtown", "Sylrob434", "stefaang", "bdoner", "cswelin", "p00chie"] 0.9999999999999993
["alphaleonis", "iislucas", "vvakame", "nestalk", "jtmueller", "mickdelaney", "staxmanade", "bluong", "martinmcwhorter", "pushplay", "smithkl42", "flcdrg", "nycdotnet", "misak113", "craigktreasure", "jpevarnek", "lokeshpeta", "xperiments", "ryiwamoto", "SergioMorchon", "kanreisa"

...

["afc163", "yesmeck", "TossShinHwa", "viztor", "shenqihui", "int64ago", "taviroquai", "williamkhshea", "abenhamdine", "RaoHai", "kirbo", "kagd", "djyde", "Alex1990", "miaojiuchen", "picodoth", "tvasenin", "benjycui", "plandem", "snadn", "ystarlongzi", "matthewlein", "timarney", "manjitkumar", "cdeutsch", "zheeeng", "queimadus", "madisvain", "JesperWe", "wuzhao", "chentony", "yutingzhao1991", "jaredleechn", "muzea", "Kamahl19", "pnghai", "jackple", "LeuisKen", "ChrisFan", "MrPeak", "edgji", "sosz", "ddcat1115", "mzohaibqc", "novi", "koobitor", "lixiaoyang1992", "kossel", "Attrash-Islam", "wSLecHayfIeNdock", "aaronplanell", "Nokecy", "sjy", "khalibloo", "dsgkirkby", "ziaochina", "farskid", "Frezc", "diegourban", "menberg", "KgTong", "valleykid", "acamposruiz", "Rohanhacker", "mgrdevport", "twisger", "blkdr", "chj-damon", "hengkx", "LeeHarlan", "kocielnik", "weihong1028", "delesseps", "danedavid", "yociduo", "ahalimkara", "dilidili", "costagolub", "kanweiwei", "EscoCreeks", "happier2", "j

["ataraxus", "sankargorthi", "log0ymxm", "chubinou", "JackWink", "wonderslug", "soulfly", "ferristseng", "philicious", "bluengreen", "Glagnar", "wing328", "ianjoneill", "RobertBiehl", "hideya", "toomuchpete", "beatcracker", "dpeschman", "aersamkull", "kartenkarsten", "y-higuchi", "brianvoss", "frinux", "keysolutions", "nazieb", "Articus", "Edubits", "SparrowJang", "pamuche", "tomekc", "F481", "julienfouilhe", "BastiOfBerlin", "wwwjfy", "laurentvdl", "mtngld", "kolyjjj", "gwen15", "zxy198717", "shawkinsl", "ampem", "apolci", "kkrauth", "rostskadat", "ajainy", "akajla09", "jgavris", "justusth", "thibaultclem", "pculligan", "rbpasker", "jaz-ah", "davebaird", "woz5999", "mtakaki", "ww-mgr", "sugi", "GriffinSchneider", "jerry1621", "dhontecillas", "bmsantos", "ahmettahasakar", "wolfAle", "giovannibotta", "joeboyscout04", "neoasket", "vzeman", "nrekretep", "abcsun", "mmschettler", "JMDHo", "hcwilhelm", "kesh92", "andriydruk", "dmytrobr", "mbarclay", "zhenjun115", "kim-sondrup", "qtstc", "sal

["wobh", "kytrinyx", "tstirrat15", "masters3d", "hankturowski", "allcentury", "LegalizeAdulthood", "drueck", "simonjefford", "jendiamond", "mdesantis", "brettchalupa", "RickWieman", "seejohnrun", "ohookins", "ZacharyRSmith", "lhoBas", "snahor", "spazm", "scottwhudson", "groyoh", "sit", "nickkjordan", "vguerra", "davearonson", "aarti", "connorlay", "bcc32", "ebengt", "amar47shah", "kickinbahk", "DuBistKomisch", "sarahjschultz", "mike-hewitson", "claytonflesher", "junedev", "AJEzk", "laurafeier", "ecrmnn", "oalbe", "AlexWheeler", "amscotti", "ramyaravindranath", "sdavids13", "betegelse", "sandhub", "drewbs", "JPBetley", "aranair", "CRivasGomez", "celiagomezdev", "jdsutherland", "thedevelopnik", "Rahmeen14", "m-sameer", "kchenery", "yilkalargaw"] 0.9999999999999987
["orta", "iosdev-republicofapps", "soffes", "tamastimar", "fabiomassimo", "caffeineflo", "natan", "sgtsquiggs", "jacobwgillespie", "pbendersky", "aksswami", "michalkonturek", "AndyChenW", "rbsgn", "rhodgkins", "tabuchid", "imak

["jaroslawr", "tarsius", "proofit404", "ahungry", "shackra", "katspaugh", "kiwanami", "eschulte", "holocronweaver", "myuhe", "bsuh", "takaxp", "gabesoft", "johanclaesson", "osener", "uu1101", "garyo", "dangirsh", "bastianbeischer", "baohaojun", "ytaka", "iRi-E", "khiker", "xwl", "rubikitch", "decoursin", "loveshack", "BinaryKhaos", "larsmagne", "naclander", "swflint", "jtatarik", "dellison", "dajva"] 0.9999999999999988
["x64x6a", "abpolym", "vvvverre", "pawlos", "boogy", "niklasb", "dhanvi", "wapiflapi", "bcyrill", "peter50216", "evandrix", "denwilly", "redknight99", "cryptax", "tyage", "superkojiman", "RyanAiden", "derlaft", "mrexcessive", "mrt-prodz", "gaffe23", "stefan2904", "ZetaTwo", "boaster", "mheistermann", "tomvangoethem", "fbonzon", "elliptic-shiho", "pe3zx", "JonathanSinger", "f0rki", "unicornsasfuel", "LiveOverflow", "chadillac", "jagotu", "L4ys", "ctfhacker", "vito-lbs", "x0w1", "tothi", "Matir"] 0.9999999999999988
["wm4", "bucaneer", "Nikoli", "quilloss", "teohhanhui", "B

["oppianmatt", "jwfang", "taion809", "bndw", "jhspaybar", "codesuki", "jsedgwick", "PiotrSikora", "junr03", "rshriram", "moderation", "JonathanO", "tonya11en", "winguse", "kavyako", "mattklein123", "misterwilliam", "lizan", "fengli79", "htuch", "glicht", "adunham-stripe", "mattwoodyard", "rlazarus", "tschroed", "hennna", "dnoe", "ramaraochavali", "mrice32", "jrajahalme", "alyssawilk", "baranov1ch", "mtakigiku", "saumoh", "rlenglet", "kmyerson", "ggreenway", "akonradi", "shakti-das", "cpakulski", "talnordan"] 1.0009765625
["Zrachel", "luotao1", "dayhaha", "beckett1124", "llxxxll", "water1981"] 1.0009765625
["alvations", "wangkuiyi", "reyoung", "drinktee", "AoZhang", "backyes", "pengli09", "luotao1", "zhouxiao-coder", "5idaidai", "wen-bo-yang", "tizhou86", "hohdiy", "Shi-Liang", "zzsu", "pkuyym", "skylarch", "seiriosPlus"] 1.0009765625
["velconia", "luotao1", "lcy-seso", "water1981", "chrisxu2016", "juliecbd", "cxysteven", "wwhu", "guoshengCS", "comeonfox", "ranqiu92"] 1.0009765625
["abo